Preprocessing of the [Book-Crossing Dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). Rational for some steps comes from [dataset exploration](https://github.com/katarinagresova/MLprojects/blob/main/BookRecommendations/playground.ipynb) notebok.

# Notes

- it could be interesting to add feature *language* to books - there are many different languages in dataset

# Setup

In [1]:
!wget http://www2.informatik.uni-freiburg.de/~cziegler/BX/BX-CSV-Dump.zip

--2021-11-07 18:54:14--  http://www2.informatik.uni-freiburg.de/~cziegler/BX/BX-CSV-Dump.zip
Resolving www2.informatik.uni-freiburg.de (www2.informatik.uni-freiburg.de)... 132.230.105.133
Connecting to www2.informatik.uni-freiburg.de (www2.informatik.uni-freiburg.de)|132.230.105.133|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘BX-CSV-Dump.zip.2’

BX-CSV-Dump.zip.2   100%[===================>]  24.88M  11.6MB/s    in 2.1s    

2021-11-07 18:54:17 (11.6 MB/s) - ‘BX-CSV-Dump.zip.2’ saved [26085508/26085508]



In [2]:
!unzip BX-CSV-Dump.zip

Archive:  BX-CSV-Dump.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace BX-Users.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [3]:
!pip install isbnlib

In [4]:
import pandas as pd
import isbnlib
from tqdm import tqdm

In [31]:
users = pd.read_csv('BX-Users.csv', sep=';', encoding = "ISO-8859-1")
books = pd.read_csv('BX-Books.csv', sep=';', encoding = "ISO-8859-1", escapechar = "\\")
ratings = pd.read_csv('BX-Book-Ratings.csv', sep=';', encoding = "ISO-8859-1")

# Books table

## Data cleaning

### Useless columns

Let's look at columns if there are some useful information.

In [6]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


We might use image in final application to show a preview of sugested books to a user. Or we could use it in multi-modal model as one of input for computing similarity between books. But for now, we will not need it, since we will do just simple proof-of-concept recommendation system.

In [32]:
books = books.drop(columns=['Image-URL-S', 'Image-URL-M', 'Image-URL-L'])

In [8]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


### Transforming ISBN to cannonical form

I found this nice python package called [isbnlib](https://github.com/xlcnd/isbnlib) that can be used to validate, clean, transform, hyphenate and get metadata for ISBN strings.

Reasons for bothering with ISBN numbers:
- transform ISBN numbers into standard form to prevent duplicate entries
- use ISBN as unique and valid identificator of a book, so we could connect it with other resources where books are identified by ISBN number.


In [9]:
books_count = len(books)
ratings_count = len(ratings)
unique_books = len(books['ISBN'].unique())
unique_ratings = len(ratings['ISBN'].unique())

print("Books count: ", books_count)
print("Books count in ratings: ", ratings_count)
print("Unique books count: ", unique_books)
print("Unique books in ratings: ", unique_ratings)

Books count:  271379
Books count in ratings:  1149780
Unique books count:  271379
Unique books in ratings:  340556


In [33]:
books['ISBN'] = books['ISBN'].apply(lambda x: isbnlib.canonical(isbnlib.clean(x)))
ratings['ISBN'] = ratings['ISBN'].apply(lambda x: isbnlib.canonical(isbnlib.clean(x)))

In [12]:
print("Removed books: ", books_count - len(books))
print("Removed books in ratings: ", ratings_count - len(ratings))
print("Removed unique books: ", unique_books - len(books['ISBN'].unique()))
print("Removed unique books in ratings: ", unique_ratings - len(ratings['ISBN'].unique()))

books_count = len(books)
ratings_count = len(ratings)
unique_books = len(books['ISBN'].unique())
unique_ratings = len(ratings['ISBN'].unique())

Removed books:  0
Removed books in ratings:  0
Removed unique books:  431
Removed unique books in ratings:  8134


As we can see, after trasforming to cannonical form, we have less unique ISBN numbers.

### Removing invalid ISBN

In [34]:
invalid_ISBN = {
                *set(books['ISBN'][books['ISBN'].apply(lambda x: (not isbnlib.is_isbn10(x)) and (not isbnlib.is_isbn13(x)))]),
                *set(ratings['ISBN'][ratings['ISBN'].apply(lambda x: (not isbnlib.is_isbn10(x)) and (not isbnlib.is_isbn13(x)))])
}
print(len(invalid_ISBN))
print(list(invalid_ISBN)[:10])

4045
['', '8817333931', '2859401330', '0088214479', '3959222411', '7116200599', '0240280553', '0440195934495', '0879575407', '0449006186']


Empty ISBN is result of trying to transform ASIN (Amazon ID) to correct ISBN numbers. There we few ASIN numbers in original data and there is no straightforward way how to transform them to ISBN (except for mining Amazon websites).

Some of invalid ISBN could be manualy fixed - there are typos.


In [37]:
books['ISBN'].apply(lambda x: x in invalid_ISBN).value_counts()

False    271262
True        117
Name: ISBN, dtype: int64

In [38]:
books = books[books['ISBN'].apply(lambda x: x not in invalid_ISBN)]
ratings = ratings[ratings['ISBN'].apply(lambda x: x not in invalid_ISBN)]

In [16]:
print("Removed books: ", books_count - len(books))
print("Removed books in ratings: ", ratings_count - len(ratings))
print("Removed unique books: ", unique_books - len(books['ISBN'].unique()))
print("Removed unique books in ratings: ", unique_ratings - len(ratings['ISBN'].unique()))

books_count = len(books)
ratings_count = len(ratings)
unique_books = len(books['ISBN'].unique())
unique_ratings = len(ratings['ISBN'].unique())

Removed books:  117
Removed books in ratings:  14593
Removed unique books:  1
Removed unique books in ratings:  4045


### Deduplicating

- for books: drop duplicates
- for ratings: duplicates have been merged by transforming to cannonical forms

In [39]:
books = books.drop_duplicates()

In [18]:
print("Removed books: ", books_count - len(books))
print("Removed books in ratings: ", ratings_count - len(ratings))
print("Removed unique books: ", unique_books - len(books['ISBN'].unique()))
print("Removed unique books in ratings: ", unique_ratings - len(ratings['ISBN'].unique()))

books_count = len(books)
ratings_count = len(ratings)
unique_books = len(books['ISBN'].unique())
unique_ratings = len(ratings['ISBN'].unique())

Removed books:  314
Removed books in ratings:  0
Removed unique books:  0
Removed unique books in ratings:  0


## Data completing

### Missing books records

In [19]:
ratings['ISBN'][~ratings['ISBN'].isin(books['ISBN'])]

6          3257224281
7          0600570967
25         3442437407
26         033390804X
30         8440682697
              ...    
1149442    0330337297
1149444    0747252327
1149446    8804413530
1149581    0679781587
1149758    8445072897
Name: ISBN, Length: 103780, dtype: object

We have ratings for 103 780 books that are not in *books* dataset. I created following function for looking up book metadata via `isbnlib` package. But it is not supporting batch request so it would take more then 30h to run.

In [20]:
def find_lost_books():

  new_books = []
  not_found = []
  for missing_isbn in tqdm(ratings['ISBN'][~ratings['ISBN'].isin(books['ISBN'])]):

    try:
      book = isbnlib.meta(missing_isbn, service='goob')
    except:
      try: 
        book = isbnlib.meta(missing_isbn, service='wiki')
      except:
        not_found.append(missing_isbn)
        continue

    if not book:
      not_found.append(missing_isbn)
      continue
    isbn_id = 'ISBN-13' if 'ISBN-13' in book.keys() else 'ISBN-10'
    new_books.append([
        book[isbn_id],
        book['Title'],
        book['Authors'][0],
        book['Year'],
        book['Publisher']
    ])
    
  return new_books, not_found

In [21]:
# do this if you have more then 30 hours
#new_books, not_found = find_lost_books()
#books.append(pd.DataFrame(new_books, columns=books.columns))
#print(not_found)

In [42]:
# for now, just drop rating without record in books
atings = ratings[ratings['ISBN'].isin(books['ISBN'])]

In [43]:
print("Removed books: ", books_count - len(books))
print("Removed books in ratings: ", ratings_count - len(ratings))
print("Removed unique books: ", unique_books - len(books['ISBN'].unique()))
print("Removed unique books in ratings: ", unique_ratings - len(ratings['ISBN'].unique()))

books_count = len(books)
ratings_count = len(ratings)
unique_books = len(books['ISBN'].unique())
unique_ratings = len(ratings['ISBN'].unique())

Removed books:  0
Removed books in ratings:  103780
Removed unique books:  0
Removed unique books in ratings:  58630


### Missing book metadata

In [22]:
books.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270948 entries, 0 to 271378
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 270948 non-null  object
 1   Book-Title           270948 non-null  object
 2   Book-Author          270947 non-null  object
 3   Year-Of-Publication  270948 non-null  int64 
 4   Publisher            270946 non-null  object
dtypes: int64(1), object(4)
memory usage: 12.4+ MB


There is one record with missing author, so we might try to look it up by hand.

In [49]:
books[books['Book-Author'].isnull()]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
187700,9627982032,The Credit Suisse Guide to Managing Your Perso...,NaN,1995,Edinburgh Financial Publishing


In [51]:
isbnlib.meta('9627982032', service='wiki')

{'Authors': ['Larissa Downes', 'Crédit suisse'],
 'ISBN-13': '9789627982036',
 'Language': '',
 'Publisher': 'Edinburgh Financial Pub. (Asia)',
 'Title': 'The Credit Suisse guide to managing your personal wealth',
 'Year': '1995'}

In [56]:
books['Book-Author'][187700] = 'Larissa Downes'

In [57]:
books[books['Book-Author'].isnull()]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher


In [23]:
books.describe()

,Year-Of-Publication
count,270948.000000
mean,1960.156381
std,256.505194
min,0.000000
25%,1989.000000
50%,1995.000000
75%,2000.000000
max,2050.000000


In [61]:
books[books['Year-Of-Publication'].apply(lambda x: x < 1900 or x > 2010)]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
176,3150000335,Kabale Und Liebe,Schiller,0,"Philipp Reclam, Jun Verlag GmbH"
188,342311360X,Die Liebe in Den Zelten,Gabriel Garcia Marquez,0,Deutscher Taschenbuch Verlag (DTV)
288,0571197639,Poisonwood Bible Edition Uk,Barbara Kingsolver,0,Faber Faber Inc
351,3596214629,"Herr Der Fliegen (Fiction, Poetry and Drama)",Golding,0,Fischer Taschenbuch Verlag GmbH
542,8845229041,Biblioteca Universale Rizzoli: Sulla Sponda De...,P Coelho,0,Fabbri - RCS Libri
...,...,...,...,...,...
270813,014029953X,Foe (Essential.penguin S.),J.M. Coetzee,0,Penguin Books Ltd
270932,0340571187,Postmens House,Maggie Hemingway,0,Trafalgar Square
271113,8427201079,El Misterio De Sittaford,Agatha Christie,0,Editorial Molino
271201,0887781721,Tom Penny,Tony German,0,P. Martin Associates


With bit of time we might try to lookup years of publication.

In [62]:
def find_lost_years():

  not_found = []
  for index in tqdm(books[books['Year-Of-Publication'].apply(lambda x: x < 1900 or x > 2010)].index):

    try:
      book = isbnlib.meta(books['ISBN'][index], service='goob')
    except:
      continue

    if not book:
      not_found.append(index)
      continue

    if 'Year' in books.keys():
      books['Year-Of-Publication'][index] = book['Year']
    else:
      not_found.append(index)
    
  return not_found

In [ ]:
#not_found = find_lost_years()
#print(not_found)